# RDDの操作を設定する

[Introduction to Spark with Python, by Jose A. Dianes](https://github.com/jadianes/spark-py-notebooks)

Sparkは、RDD自体が適切に設定されていない場合でも、組合や交差点などの数学的なセットでの操作の多くをサポートしています。 これらの操作は、操作されているRDDが同じタイプであることを必要とすることに注意することが重要である。


セット操作は、期待どおりに動作するので、理解するのが非常に簡単です。 唯一の考慮事項は、RDDが実際のセットではないという事実から来ており、したがってRDDの組合などの操作は重複を除去しない。 このノートでは、減算、区別、およびデカルトを簡単に見ていきます。

## データの取得とRDDの作成

最初のノートブックで行ったように、KDD Cup 1999に提供された縮小データセット（10％）を使用し、約50万のネットワークインタラクションが含まれます。このファイルは、ローカルでダウンロードするGzipファイルとして提供されています。


In [1]:
import urllib
f = urllib.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

In [2]:
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

## 「減算」を使用して攻撃のやりとりを取得する Getting attack interactions using `subtract` 

説明の目的のために、以前の分析からの非攻撃（正常）の相互作用を伴うRDDをすでに持っていると想像してください。
For illustrative purposes, imagine we already have our RDD with non attack (normal) interactions from some previous analysis.   

In [3]:
normal_raw_data = raw_data.filter(lambda x: "normal." in x)

以下のように、元のフィルタリングされていないRDDから通常のものを減算することによって、攻撃相互作用を得ることができる。
We can obtain attack interactions by subtracting normal ones from the original unfiltered RDD as follows.  

In [4]:
attack_raw_data = raw_data.subtract(normal_raw_data)

私たちの結果を確認するためにいくつかのカウントをしましょう。

In [5]:
from time import time

# count all
t0 = time()
raw_data_count = raw_data.count()
tt = time() - t0
print "All count in {} secs".format(round(tt,3))

All count in 3.359 secs


In [6]:
# count normal
t0 = time()
normal_raw_data_count = normal_raw_data.count()
tt = time() - t0
print "Normal count in {} secs".format(round(tt,3))

Normal count in 2.203 secs


In [7]:
# count attacks
t0 = time()
attack_raw_data_count = attack_raw_data.count()
tt = time() - t0
print "Attack count in {} secs".format(round(tt,3))

Attack count in 8.992 secs


In [8]:
print "There are {} normal interactions and {} attacks, \
from a total of {} interactions".format(normal_raw_data_count,attack_raw_data_count,raw_data_count)

There are 97278 normal interactions and 396743 attacks, from a total of 494021 interactions


だから今私たちは2つのRDDを持っています.1つは正常なやりとりと、もう1つは攻撃ですSo now we have two RDDs, one with normal interactions and another one with attacks.  

## デカルトを使用したプロトコルとサービスの組み合わせ Protocol and service combinations using `cartesian`

デカルト変換を使用して、2つのRDD間のデカルト積を計算することができます。 これは、2つのRDDの間にあるすべての可能な要素のペアを返します。 私たちの場合、私たちはネットワークインタラクションのサービスとプロトコルの間のすべての可能な組み合わせを生成するためにそれを使用します。

まず、2つの別々のRDDで各値の集合を分離する必要があります。 そのためには、CSVで解析されたデータセットで `distinct`を使用します。 [データセットの説明]（http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names）から、プロトコルは2番目の列であり、サービスは3番目であることがわかります（タグは最初のものではなく最後のものです このページにあるように）。

だから最初に、プロトコルを手に入れましょう。

In [9]:
csv_data = raw_data.map(lambda x: x.split(","))
protocols = csv_data.map(lambda x: x[1]).distinct()
protocols.collect()

[u'udp', u'icmp', u'tcp']

Now we do the same for services.  

In [10]:
services = csv_data.map(lambda x: x[2]).distinct()
services.collect()

[u'domain',
 u'http_443',
 u'Z39_50',
 u'smtp',
 u'urp_i',
 u'private',
 u'echo',
 u'shell',
 u'red_i',
 u'eco_i',
 u'sunrpc',
 u'ftp_data',
 u'urh_i',
 u'pm_dump',
 u'pop_3',
 u'pop_2',
 u'systat',
 u'ftp',
 u'uucp',
 u'whois',
 u'netbios_dgm',
 u'efs',
 u'remote_job',
 u'daytime',
 u'ntp_u',
 u'finger',
 u'ldap',
 u'netbios_ns',
 u'kshell',
 u'iso_tsap',
 u'ecr_i',
 u'nntp',
 u'printer',
 u'domain_u',
 u'uucp_path',
 u'courier',
 u'exec',
 u'time',
 u'netstat',
 u'telnet',
 u'gopher',
 u'rje',
 u'sql_net',
 u'link',
 u'auth',
 u'netbios_ssn',
 u'csnet_ns',
 u'X11',
 u'IRC',
 u'tftp_u',
 u'login',
 u'supdup',
 u'name',
 u'nnsp',
 u'mtp',
 u'http',
 u'bgp',
 u'ctf',
 u'hostnames',
 u'klogin',
 u'vmnet',
 u'tim_i',
 u'discard',
 u'imap4',
 u'other',
 u'ssh']

この場合は長いリストです。

今、我々はデカルト積を行うことができます。

In [11]:
product = protocols.cartesian(services).collect()
print "There are {} combinations of protocol X service".format(len(product))

There are 198 combinations of protocol X service


明らかに、このような小さなRDDでは、Sparkデカルト積を使用するのは実際には意味がありません。 我々はdistinctを使った後に値を完全に収集し、デカルト積を局所的に行うことができた。 さらに、個別およびデカルトは高価な操作であるため、操作データセットが大きい場合には注意して使用する必要があります。